In [1]:
import pandas as pd
import datasets
from transformers import BertTokenizer
import numpy as np

In [2]:
concatenated = pd.read_csv("../data/brazilian_portuguese_sentiment/concatenated.csv")

/tmp/ipykernel_14948/3934628482.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  concatenated = pd.read_csv("../data/brazilian_portuguese_sentiment/concatenated.csv")


In [3]:
concatenated

,dataset,original_index,review_text,review_text_processed,review_text_tokenized,polarity,rating,kfold_polarity,kfold_rating
0,b2w,11955,Bem macio e felpudo...recomendo. Preço imbatí...,bem macio e felpudo...recomendo. preco imbati...,"['bem', 'macio', 'felpudo', 'recomendo', 'prec...",1.0,4.0,1,1
1,b2w,35478,Produto excepcional! recomendo!!! inovador e ...,produto excepcional! recomendo!!! inovador e ...,"['produto', 'excepcional', 'recomendo', 'inova...",1.0,5.0,1,1
2,b2w,122760,recebi o produto antes do prazo mas veio com d...,recebi o produto antes do prazo mas veio com d...,"['recebi', 'produto', 'antes', 'do', 'prazo', ...",0.0,1.0,1,1
3,b2w,17114,Bom custo beneficio. Adequado para pessoas que...,bom custo beneficio. adequado para pessoas que...,"['bom', 'custo', 'beneficio', 'adequado', 'par...",1.0,5.0,1,1
4,b2w,19112,Além de higiênico tem o tamanho ideal. Só falt...,alem de higienico tem o tamanho ideal. so falt...,"['alem', 'de', 'higienico', 'tem', 'tamanho', ...",NaN,3.0,-1,1
...,...,...,...,...,...,...,...,...,...
2786087,buscape,1_422965,"Produto muito bom, simples e barato","produto muito bom, simples e barato","['produto', 'muito', 'bom', 'simples', 'barato']",1.0,5.0,10,10
2786088,buscape,minus_1_150466,O esquema antigo de desmontagem e limpeza das ...,o esquema antigo de desmontagem e limpeza das ...,"['esquema', 'antigo', 'de', 'desmontagem', 'li...",NaN,3.0,-1,10
2786089,buscape,0_414799,Esse jogo é muito maneiro é um jogo onde vc te...,esse jogo e muito maneiro e um jogo onde vc te...,"['esse', 'jogo', 'muito', 'maneiro', 'um', 'jo...",1.0,5.0,10,10
2786090,buscape,0_389898,Muito bom e intuitivo!\n\nO que gostei: Educa ...,muito bom e intuitivo!\n\no que gostei: educa ...,"['muito', 'bom', 'intuitivo', 'que', 'gostei',...",NaN,3.0,-1,10


In [4]:
hg_dataset = concatenated[["polarity", "review_text"]].dropna()
hg_dataset

,polarity,review_text
0,1.0,Bem macio e felpudo...recomendo. Preço imbatí...
1,1.0,Produto excepcional! recomendo!!! inovador e ...
2,0.0,recebi o produto antes do prazo mas veio com d...
3,1.0,Bom custo beneficio. Adequado para pessoas que...
5,1.0,"ótmo produto atendeu as expectativas, a entreg..."
...,...,...
2786085,1.0,Excelente custo-benefício apresentado por este...
2786086,1.0,"Game muito legal, com uma grande variedade de ..."
2786087,1.0,"Produto muito bom, simples e barato"
2786089,1.0,Esse jogo é muito maneiro é um jogo onde vc te...


In [5]:
hg_dataset = datasets.Dataset.from_pandas(hg_dataset, split="train")
hg_dataset = hg_dataset.remove_columns("__index_level_0__")

In [6]:
hg_dataset

Dataset({
    features: ['polarity', 'review_text'],
    num_rows: 2386163
})

In [7]:
#hg_dataset.push_to_hub("jvanz/portuguese_sentiment_analysis")

In [8]:
tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
hg_dataset = hg_dataset.map(lambda x:tokenizer(x["review_text"], add_special_tokens=False), num_proc=6,batched=True)
hg_dataset

#0:   0%|          | 0/398 [00:00<?, ?ba/s]

#1:   0%|          | 0/398 [00:00<?, ?ba/s]

#2:   0%|          | 0/398 [00:00<?, ?ba/s]

#3:   0%|          | 0/398 [00:00<?, ?ba/s]

#4:   0%|          | 0/398 [00:00<?, ?ba/s]

#5:   0%|          | 0/398 [00:00<?, ?ba/s]

Dataset({
    features: ['polarity', 'review_text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2386163
})

In [9]:
hg_dataset = hg_dataset.map(lambda x: {"tokens_count": len(x["input_ids"])}).filter(lambda x: x["tokens_count"] != 0)

  0%|          | 0/2386163 [00:00<?, ?ex/s]

  0%|          | 0/2387 [00:00<?, ?ba/s]

In [ ]:
dataset = hg_dataset.to_pandas()
dataset

In [ ]:
filtered_dataset = dataset[dataset.tokens_count != 0]
filtered_dataset